In [1]:
source("SimData.r")
library(MASS)
library("glmertree")
library("WGCNA")
library("pre")

Warning message:
"package 'glmertree' was built under R version 3.6.1"Loading required package: lme4
Warning message:
"package 'lme4' was built under R version 3.6.1"Loading required package: Matrix
Loading required package: partykit
Warning message:
"package 'partykit' was built under R version 3.6.1"Loading required package: grid
Loading required package: libcoin
Warning message:
"package 'libcoin' was built under R version 3.6.1"Loading required package: mvtnorm
Warning message:
"package 'WGCNA' was built under R version 3.6.1"Loading required package: dynamicTreeCut
Loading required package: fastcluster

Attaching package: 'fastcluster'

The following object is masked from 'package:stats':

    hclust


Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: 'WGCNA'

The following object is masked from 'package:stats':

    cor

Warning message:
"package 'pre' was built under R v

Problems
* When fixed_regress = NULL, we can let the user decide whether to use PC or not as regressors. If don't use PC, that is a WGCNA+ regular RE-EM
* Right now we can only do random intercept
* User may want to tune other parameters in WGCNA (in addition to power). How to do this in a elegant way? (Not urgent since the current parameters can cluster correctly)
* max_depth_screen_factor=0.04 and max_depth_select_factor = 0.8 (not urgent)
* Right now all the alpha in the algorithm are the same.

Not finished
* In Longtree function where fixed_regress ==NULL, just use PC and glmertree.

Output: a glmertree object (trained tree)

Parameters:
* data: training data
* fixed_regress: the regressors used no matter what such as time and time^2; if fixed_regress = NULL, use PC as regressor at screening step
* fixed_split: a char vector containing features definitely used in splitting
* var_select: a char vector containing features to be selected. These features will be clustered by WGCNA and the chosen ones will be used in regression and splitting
* power: parameters of WGCNA
* cluster: the variable name of each cluster (in terms of random effect)
* Fuzzy = TRUE: Screen like Fuzzy Forest; Fuzzy= FALSE: first screen within non-grey modules and then select the final non-grey features within the selected ones from each non-grey module; Use this final non-grey features as regressors (plus fixed_regress) and use grey features as split_var to select grey features. The use final non-grey features and selected grey features together in splitting and regression variables, to do the final prediction. Fuzzy=FALSE is used if there are so many non-grey features and you want to protect grey features.
* maxdepth_factor_screen: when selecting features from one module, the maxdepth of the glmertree is set to ceiling function of maxdepth_factor_screen*(#features in that module). Default is 0.04. 
* maxdepth_factor_select: Given screened features (from each modules, if Fuzzy=FALSE,that is the selected non-grey features from each non-grey modules), we want to select again from those screened features. The maxdepth of that glmertree is set to be ceiling of maxdepth_factor_select*(#screened features). Default is 0.6.
* for the maxdepth of the prediction tree (final tree), maxdepth is set to the length of the split_var (fixed+chosen ones)
* The most important parameters are alpha and maxdepth.

In [2]:
Longtree = function(data,fixed_regress=NULL,fixed_split=NULL,var_select=NULL,
                    power=6,cluster,alpha=0.2,maxdepth_factor_screen=0.04,
                    maxdepth_factor_select=0.5,Fuzzy=TRUE){
    ### if there are no features to select, just use fixed_regress and fixed_split
    if(length(var_select)==0){
        if (length(fixed_regress)==0){
            if (length(fixed_split)==0){
                stop("no features to split and regress on")
            }
            fixed_regress = "1"
        }
        maxdepth = length(fixed_split)
        Formula = as.formula(paste("y~",paste(fixed_regress,collapse = "+"),
                                       "|",cluster,"|",
                                     paste(fixed_split,collapse = "+")))
        mytree = lmertree(Formula, data = data,alpha = alpha,maxdepth=maxdepth)
        return (mytree)
    } ###
    # Now var_select is not empty
    # If don't specify fixed_regress: use PC as regressors at screening step
    if (length(fixed_regress)==0){
        ...
        return
    }###
    # Now we have non-empty var_select,fixed_regress
    return(Longtree_time(data=data,fixed_regress=fixed_regress,
                        fixed_split=fixed_split, var_select=var_select,
                    power=power,cluster=cluster,alpha=alpha,
                    maxdepth_factor_screen=maxdepth_factor_screen, 
                    maxdepth_factor_select=maxdepth_factor_select,Fuzzy=Fuzzy))
                        
}

# Longtree_time: used when var_select and fixed_regress are non-empty
# Longtree is equivalent to this Longtree_time in this case
Longtree_time = function(data,fixed_regress,fixed_split,var_select,power,cluster,
                         alpha,maxdepth_factor_screen,maxdepth_factor_select,Fuzzy){
    # Cluster var_select
    data_WGCNA = data[var_select]
    # Must set numericLabels = FALSE so that it uses actual colors like "grey"
    net = blockwiseModules(data_WGCNA, power = power,TOMType = "unsigned", 
                           minModuleSize = 30,reassignThreshold = 0, 
                           mergeCutHeight = 0.25,numericLabels = FALSE, 
                           pamRespectsDendro = FALSE,verbose = 0)
    # the correspondance betweeen feature names and colors
    colors = net$colors # it is a string vector with names (that is the name is V1)
    module_names = unique(colors) # all color names
    #"dictionary"with keys=name of color,values=names of features of that color
    module_dic = list() 
    for (i in 1:length(module_names)){
        module_dic[[module_names[i]]] = names(colors[colors==module_names[i]])
    }
    
    imp_var = list() # used to store the names of important features
    
    if(Fuzzy==TRUE){
        # Do the selection step just like Fuzzy Forest:
        # for each module (including grey), use them as split_var and select
        # finally, use all selected ones as split_var and select
        
        for (name in module_names){
        # in the formula, add fixed_split as split_var, also include the module features
        split_var = c(module_dic[[name]],fixed_split)
        maxdepth = ceiling(maxdepth_factor_screen*length(split_var))
        # use fixed_regress as regressor
        regress_var = fixed_regress

        # Formula for lmtree
        Formula = as.formula(paste("y~",paste(regress_var,collapse = "+"),
                               "|",cluster,"|",
                    paste(split_var,collapse = "+")))

        # fit the tree
        mytree = lmertree(Formula, data = data,alpha = alpha,maxdepth=maxdepth) 

        #extract important features
        imp_var[[length(imp_var)+1]] = get_split_names(mytree$tree,data)
        }
        
        # the variables selected from all the modules
        final_var = imp_var[[1]]
        if (length(imp_var)>1){
            # There are at least 2 modules
            for (i in 2:length(imp_var)){
            final_var = c(final_var,imp_var[[i]])
         }
            cat("after screening within modules",final_var,"\n")
            
            # the final selection among all the chosen features 
            regress_var = fixed_regress
            split_var = c(final_var,fixed_split)
            maxdepth = ceiling(maxdepth_factor_select*length(split_var))
            Formula = as.formula(paste("y~",paste(regress_var,collapse = "+"),
                                       "|",cluster,"|",
                                     paste(split_var,collapse = "+")))
            mytree = lmertree(Formula, data = data,alpha = alpha,maxdepth=maxdepth) 
            final_var = get_split_names(mytree$tree,data)
            cat("final features",final_var)      
        }else{
            # only grey module
            cat("There is only one module, final features",final_var)
        }
        # use the final features as split&regression variables
        split_var = c(final_var,fixed_split)
        maxdepth = length(split_var)
        regress_var = c(fixed_regress,final_var)
        Formula = as.formula(paste("y~",paste(regress_var,collapse = "+"),
                                   "|",cluster,"|",
                                 paste(split_var,collapse = "+")))
        mytree = lmertree(Formula, data = data,alpha=alpha,maxdepth=maxdepth) 
        return(mytree)           
    }
    if(Fuzzy==FALSE){
        # first do the screening and selecting in non-grey modules
        # Then use those non-grey estimated true features as regressors
        # and grey features as split_var, choose grey features and keep them
        for (name in module_names){
        if(name=="grey"){
            next
        }
        split_var = c(module_dic[[name]],fixed_split)
        maxdepth = ceiling(maxdepth_factor_screen*length(split_var))
        regress_var = fixed_regress

        # Formula for lmtree
        Formula = as.formula(paste("y~",paste(regress_var,collapse = "+"),
                               "|",cluster,"|",
                    paste(split_var,collapse = "+")))

        # fit the tree
        mytree = lmertree(Formula, data = data,alpha = alpha,maxdepth=maxdepth) 

        #extract important features
        imp_var[[length(imp_var)+1]] = get_split_names(mytree$tree,data)
        }# Now imp_var contains important features from modules that are not grey
        if(length(imp_var)==0){
            # only grey module, no other modules
            split_var = c(module_dic[["grey"]],fixed_split)
            maxdepth = ceiling(maxdepth_factor_screen*length(split_var))
            regress_var = fixed_regress
            Formula = as.formula(paste("y~",paste(regress_var,collapse = "+"),
                               "|",cluster,"|",
                    paste(split_var,collapse = "+")))
            mytree = lmertree(Formula, data = data,alpha = alpha,maxdepth=maxdepth) 
            final_var = get_split_names(mytree$tree,data)
            cat("There is only one module which is grey, final features",final_var)
        }else{
            # at least one non-grey module
            final_var = imp_var[[1]]
            # if only one non-grey module: final_var is the chosen non-grey features
            # if at least two non-grey modules:
            if (length(imp_var)>1){
                # There are at least 2 modules
                for (i in 2:length(imp_var)){
                final_var = c(final_var,imp_var[[i]])
                }
            
            cat("After screening within non-grey modules",final_var,"\n")
            # select from selected non-grey features
            regress_var = fixed_regress
            split_var = c(final_var,fixed_split)
            maxdepth = ceiling(maxdepth_factor_select*length(split_var))
            Formula = as.formula(paste("y~",paste(regress_var,collapse = "+"),
                                       "|",cluster,"|",
                                     paste(split_var,collapse = "+")))
            mytree = lmertree(Formula, data = data,alpha = alpha,maxdepth=maxdepth) 
            final_var = get_split_names(mytree$tree,data)
            }
            cat("The chosen non-grey features are",final_var,"\n")
            
            # use final_var (chosen non-grey features) to select grey features
            regress_var = c(fixed_regress,final_var)
            split_var = c(module_dic[["grey"]],fixed_split)
            maxdepth = ceiling(maxdepth_factor_screen*length(split_var))
            Formula = as.formula(paste("y~",paste(regress_var,collapse = "+"),
                                       "|",cluster,"|",
                                     paste(split_var,collapse = "+")))
            mytree = lmertree(Formula, data = data,alpha = alpha,maxdepth=maxdepth) 
            grey_var = get_split_names(mytree$tree,data)
            cat("The chosen grey features are",grey_var,"\n")
            # use final_var and grey_var do to the final model tree
            final_var = c(final_var,grey_var)    
            cat("final features",final_var)  
        }
        # use the final features as split&regression variables
        split_var = c(final_var,fixed_split)
        maxdepth = length(split_var)
        regress_var = c(fixed_regress,final_var)
        Formula = as.formula(paste("y~",paste(regress_var,collapse = "+"),
                                   "|",cluster,"|",
                                 paste(split_var,collapse = "+")))
        mytree = lmertree(Formula, data = data,alpha=alpha,maxdepth=maxdepth) 
        return(mytree)
    }
    
}

# Methods for extracting names of splitting features used in a tree
# tree: a tree object; data: the train or test set
get_split_names = function(tree,data){
    # path: the string that contains all the node information
    paths <- pre:::list.rules(tree, removecomplements = FALSE)
    vnames = names(data)
    # the regex for a variable
    # tomatch = paste(paste(var,"<="),"|",paste(var,">"),sep="")
    # match to tomatch in path
    tmp = vnames[sapply(sapply(vnames, FUN = function(var) grep(paste(paste(var,"<="),"|",paste(var,">"),sep=""), paths)), length) > 0]
    return (tmp)
}

# Sample Run

In [3]:
fixed_regress = c("time","time2")
fixed_split = c("treatment")
cluster = "patient"
var_select = paste("V",1:400,sep="")

In [4]:
# training data
n <- 300 # number of patients
T <-  5 # number of observations per patients

set.seed(100)

data <- sim_quad(n,T)
# add time_squared
data$time2 = (data$time)^2

# testing data
n_test <- 100 # number of patients
T <-  5 # number of observations per patients
set.seed(101)
data_test <- sim_quad(n,T)
# data_test <- sim_quad(n_test, T)
data_test$time2 = (data_test$time)^2

In [5]:
# Fuzzy=TRUE 
# alpha = 0.2, maxdepth_factor_select = 0.5 (all default)
system.time({
    mytree = Longtree(data,fixed_regress=fixed_regress,fixed_split=fixed_split,
                  var_select=var_select,cluster=cluster,Fuzzy=TRUE)
})
mean((predict(mytree,newdata=data_test)-data_test$y)**2)

boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular


after screening within modules V1 V2 V3 V14 V26 V68 V106 V154 V172 V301 V302 V303 V319 V348 


boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular


final features V1 V2 V3 V14 V26 V301 V302 V303

   user  system elapsed 
  22.85    0.39   23.57 

[1] 16.52475

In [6]:
# Fuzzy=False 
# alpha = 0.1, maxdepth_factor_select = 0.5
system.time({
    mytree = Longtree(data,fixed_regress=fixed_regress,fixed_split=fixed_split,
                  var_select=var_select,alpha=0.1,cluster=cluster,Fuzzy=FALSE)
})
mean((predict(mytree,newdata=data_test)-data_test$y)**2)

boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular


After screening within non-grey modules V1 V2 V3 V14 V26 V68 


boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular


The chosen non-grey features are V1 V2 V3 
The chosen grey features are V301 V302 V303 
final features V1 V2 V3 V301 V302 V303

   user  system elapsed 
  24.25    0.53   25.23 

[1] 14.16315